<a href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/Precomputed_dual_CHEMBERT_GPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Running the best GPS with KNN Generate**
<a href="https://colab.research.google.com/github/raphaelrubrice/BornToOverfit/blob/raph/Precomputed_CHEMBERT_GPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Colab setup**

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

Mounted at /content/drive
/content


In [3]:
!git clone https://github.com/raphaelrubrice/BornToOverfit.git
%cd BornToOverfit
!git checkout raph

Cloning into 'BornToOverfit'...
remote: Enumerating objects: 767, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 767 (delta 57), reused 45 (delta 31), pack-reused 674 (from 1)
Receiving objects: 100% (767/767), 1.08 MiB | 4.21 MiB/s, done.
Resolving deltas: 100% (522/522), done.
/content/BornToOverfit
Branch 'raph' set up to track remote branch 'raph' from 'origin'.
Switched to a new branch 'raph'


Installing requirements

In [4]:
!pip install -r data_baseline/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 58.1 MB/s eta 0:00:00


**Downloading the data**

In [5]:
!cp -r /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/data /content/BornToOverfit/data_baseline/.

## **KNN Generate**

Add Molecule cards

In [6]:
!python data_baseline/data_utils.py

BUILD + SAVE: mol_card for all graphs

[1/3] Loading: /content/BornToOverfit/data_baseline/data/train_graphs.pkl
Loaded graphs: 31008
Adding Molecule Cards..: 100% 31008/31008 [01:15<00:00, 410.30it/s]
Added/updated mol_card for: 31008
[2/3] Saving back to same location: /content/BornToOverfit/data_baseline/data/train_graphs.pkl
[3/3] Reloading to verify mol_card persisted: /content/BornToOverfit/data_baseline/data/train_graphs.pkl

--- Sample mol_card (first graph) ---
[MOL_FEATURES]
SMILES: NC1=C2N=CN([C@@H]3O[C@H](COP(=O)([O-])OP(=O)([O-])OC4O[C@H](CO)[C@@H](O)[C@H]4O)[C@@H](O)[C@H]3OP(=O)([O-])[O-])C2=NC=N1
atoms_total: 40
elements: C=15 N=5 O=17 P=3
molecular_weight: 635.265000
high_natural_abundance_atoms: 25
nuclear_spins: 1:5; 1/2:3
halogens_total: 0
metals_or_other_total: 0
formal_charge_net: -4
aromatic_atoms: 9
ring_atoms: 19
undirected_bonds: 43
bond_types: SINGLE=30 DOUBLE=3 AROMATIC=10
conjugated_bonds: 11
hybridization: SP2=13 SP3=27
chirality_centers: 7
radical_atoms: 0

Load embeddings and models

In [20]:
!cp -r /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/Cam_Raph /content/BornToOverfit/data_baseline/.

In [22]:
import os
path_to_folder = "/content/BornToOverfit/data_baseline/Cam_Raph/"
path_to_prototype = "/content/BornToOverfit/prototype/"
path_to_data = "/content/BornToOverfit/data_baseline/data/"

os.environ['SAVING_FOLDER'] = path_to_prototype + "FULL_model_real_final_pipeline_FINAL/" #"FULL_model_v3_compat_from_dual_hd256/"
os.environ['SAVING_SUBFOLDER'] = "knn_llm"
os.environ['SAVING_SFT_SUBFOLDER'] = os.environ['SAVING_SUBFOLDER'] + "_sft"
os.environ['GNN_PATH'] = path_to_folder + "model_real_final_pipeline_FINAL.pt" #"model_v3_compat_from_dual_hd256.pt"
os.environ['TRAIN_EMB_PATH'] = path_to_folder + "train_embeddings_RealChemBERT.pt"
os.environ['VAL_EMB_PATH'] = path_to_folder + "validation_embeddings_RealChemBERT.pt"

os.environ['TRAIN_GRAPH_PATH'] = path_to_data + "train_graphs.pkl"
os.environ['VAL_GRAPH_PATH'] = path_to_data + "validation_graphs.pkl"

Create merged TRAIN/VAL data

In [23]:
os.environ['DATA_FOLDER'] = "/content/BornToOverfit/data_baseline/data"
os.environ['MERGED_GRAPHS'] = os.environ['DATA_FOLDER'] + "/train_val_graphs.pkl"
!python data_baseline/merge_data.py \
  --train_path $TRAIN_GRAPH_PATH \
  --val_path $VAL_GRAPH_PATH \
  --output_dir $DATA_FOLDER

Loading graphs from /content/BornToOverfit/data_baseline/data/train_graphs.pkl...
 -> Loaded 31008 graphs.
Loading graphs from /content/BornToOverfit/data_baseline/data/validation_graphs.pkl...
 -> Loaded 1000 graphs.
Max Train ID found: 31007. Offset calculated: 31008
Re-indexing 1000 validation graphs with offset +31008...

Merged 31008 (train) + 1000 (val) = 32008 total graphs.
Success: All merged IDs are unique.
Saving merged dataset to /content/BornToOverfit/data_baseline/data/train_val_graphs.pkl...
Done.


In [24]:
os.environ['MERGED_EMB'] = os.environ['DATA_FOLDER'] + "/train_val_embeddings.pt"
!python data_baseline/merge_embeddings.py \
  --train_path $TRAIN_EMB_PATH \
  --val_path $VAL_EMB_PATH \
  --output_dir $DATA_FOLDER

Loading embeddings from /content/BornToOverfit/data_baseline/Cam_Raph/train_embeddings_RealChemBERT.pt...
 -> Loaded 31008 embeddings.
Loading embeddings from /content/BornToOverfit/data_baseline/Cam_Raph/validation_embeddings_RealChemBERT.pt...
 -> Loaded 1000 embeddings.
Max Train ID found: 31007. Offset calculated: 31008
Re-indexing 1000 validation embeddings with offset +31008...
Merged 31008 + 1000 = 32008 total embeddings.
Saving 32008 embeddings to /content/BornToOverfit/data_baseline/data/train_val_embeddings.pt...
Done.


Use all data ?

In [25]:
USE_ALL_DATA = True
if USE_ALL_DATA:
    os.environ['TRAINING_DATA'] = os.environ['MERGED_EMB']
else:
    os.environ['TRAINING_DATA'] = os.environ['TRAIN_EMB_PATH']

Fine-tune the LLM with SFT

In [26]:
os.environ['STEPS'] = '4000'
os.environ['MAX_LENGTH'] = '2048'

In [27]:
!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_graph_path $MERGED_GRAPHS \
  -train_emb_path $TRAINING_DATA \
  --do_finetune \
  --do_sft \
  --k 5 \
  --use_lora \
  --sft_steps $STEPS \
  --sft_lora 128 \
  --sft_neftune \
  --per_device_bs 1 \
  --grad_accum 8 \
  --base_llm deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
  --out_llm_dir $SAVING_SUBFOLDER \
  --max_prompt_length $MAX_LENGTH

2026-01-13 16:10:10.813786: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-13 16:10:10.831187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768320610.851940   12571 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768320610.858348   12571 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768320610.874428   12571 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Generate on test (no references; no evaluation)

In [28]:
!python prototype/knn_tanimoto_generate.py -f_data data_baseline/data -f $SAVING_FOLDER \
  -gnn_path $GNN_PATH \
  -train_graph_path $MERGED_GRAPHS \
  -train_emb_path $TRAINING_DATA \
  --do_generate --split test --k 5 \
  --gen_batch_size 8 \
  --out_llm_dir $SAVING_SFT_SUBFOLDER \
  --max_prompt_length $MAX_LENGTH

2026-01-13 16:29:00.015997: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-13 16:29:00.034383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768321740.056059   17320 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768321740.062702   17320 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768321740.079627   17320 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

See generated data on the test set

In [29]:
import pandas as pd
os.environ['FINAL_SAVING_FOLDER'] = os.environ['SAVING_FOLDER'][:-1]
test = pd.read_csv(os.environ['FINAL_SAVING_FOLDER'] + "/test_knn_gen_k5.csv")
test

FileNotFoundError: [Errno 2] No such file or directory: '/content/BornToOverfit/prototype/FULL_model_real_final_pipeline_FINAL/test_knn_gen_k5.csv'

In [ ]:
print(test.iloc[18]["Prompt"])
print(test.iloc[18]["generated_description"])

Saving to Drive

In [ ]:
!cp -r $FINAL_SAVING_FOLDER /content/drive/MyDrive/MVA/ALTEGRAD/challenge_data/.